In [98]:
import os
import sys
stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')
import keras
sys.stderr = stderr

import pandas as pd

## Preparación del dataframe

### Elimino NAN

Me quedo con las columnas que me interesan basándome en la importancia medida con univariate.

In [99]:
TARGET = 'precio'
columnas = ["tipodepropiedad", "ciudad", "antiguedad", "habitaciones", "garages", "banos", "metroscubiertos",
           "metrostotales", "gimnasio", "usosmultiples", "piscina", "escuelascercanas", "centroscomercialescercanos", "precio"]

In [100]:
train = pd.read_csv('sets_de_datos/train.csv')

In [101]:
train = train[columnas]
train["antiguedad"].fillna(train["antiguedad"].mean(),inplace=True)
train["metroscubiertos"].fillna(train["metroscubiertos"].mean(), inplace=True)
train["metrostotales"].fillna(train["metrostotales"].mean(), inplace=True)
train["precio"].fillna(train["precio"].mean(), inplace=True)
train.dropna(subset=["tipodepropiedad", "ciudad"], inplace=True)
train.fillna(0, inplace=True)

### Transformo las variables de tipo de propiedad y ciudad en numéricas con OHE

In [102]:
ciudades_validas = train["ciudad"].value_counts().to_frame().reset_index()
ciudades_validas.columns = ["ciudad", "cantidad"]
ciudades_validas.shape

(875, 2)

In [103]:
# Elimino las ciudades con menos de 100 casas para reducir la cantidad de col cuando se haga OHE
ciudades_validas = ciudades_validas[ciudades_validas["cantidad"] >= 100]
train = pd.merge(train, ciudades_validas, on = "ciudad", how = "inner")
train = train.drop(["cantidad"], axis = 1)

### Split del set de datos

In [104]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

In [105]:
train  = pd.get_dummies(train)

In [106]:
train.shape

(232962, 182)

In [107]:
X = train.drop([TARGET], axis = 1).copy()
Y = train[TARGET].copy().values

min_max_scaler = preprocessing.MinMaxScaler()
X = min_max_scaler.fit_transform(X)

X_train, X_val_and_test, Y_train, Y_val_and_test = train_test_split(X, Y, test_size=0.3, random_state=42)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_and_test, Y_val_and_test, test_size=0.5, random_state=42)

## Creación de la neural network

In [108]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense

In [109]:
model = Sequential([
    Dense(32, activation='relu', input_shape=(182 - 1,)),
    Dense(32, activation='relu'),
    Dense(1, activation='softmax'),
])

In [110]:
model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=['mean_squared_error'])

In [112]:
hist = model.fit(X_train, Y_train,
          batch_size=32, epochs=100,
          validation_data=(X_val, Y_val))

Train on 163073 samples, validate on 34944 samples
Epoch 1/100
163073/163073 [==============================] - 10s 63us/step - loss: 11176511611831.2031 - mean_squared_error: 11176511611831.2031 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 2/100
163073/163073 [==============================] - 10s 60us/step - loss: 11176511599163.9023 - mean_squared_error: 11176511599163.9023 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 3/100
163073/163073 [==============================] - 10s 62us/step - loss: 11176511615746.3320 - mean_squared_error: 11176511615746.3320 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 4/100
163073/163073 [==============================] - 14s 84us/step - loss: 11176511606053.7559 - mean_squared_error: 11176511606053.7559 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 5/100
163073/163073 [==============================

163073/163073 [==============================] - 10s 59us/step - loss: 11176511588772.8574 - mean_squared_error: 11176511588772.8574 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 38/100
163073/163073 [==============================] - 10s 62us/step - loss: 11176511605275.7148 - mean_squared_error: 11176511605275.7148 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 39/100
163073/163073 [==============================] - 10s 60us/step - loss: 11176511606648.5410 - mean_squared_error: 11176511606648.5410 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 40/100
163073/163073 [==============================] - 10s 64us/step - loss: 11176511602880.5020 - mean_squared_error: 11176511602880.5020 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 41/100
163073/163073 [==============================] - 10s 64us/step - loss: 11176511595916.7012 - mean_square

163073/163073 [==============================] - 10s 64us/step - loss: 11176511588242.3750 - mean_squared_error: 11176511588242.3750 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 74/100
163073/163073 [==============================] - 10s 62us/step - loss: 11176511613921.5898 - mean_squared_error: 11176511613921.5898 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 75/100
163073/163073 [==============================] - 9s 58us/step - loss: 11176511605803.3848 - mean_squared_error: 11176511605803.3848 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 76/100
163073/163073 [==============================] - 10s 61us/step - loss: 11176511599716.8906 - mean_squared_error: 11176511599716.8906 - val_loss: 11342327631917.0117 - val_mean_squared_error: 11342327631917.0117
Epoch 77/100
163073/163073 [==============================] - 10s 63us/step - loss: 11176511594662.0273 - mean_squared

In [113]:
model.evaluate(X_test, Y_test)[1]

34945/34945 [==============================] - 1s 28us/step


11123648602120.205